In [1]:
import pandas as pd
from Bio import pairwise2

In [2]:
# TODO: merge D1 and D2
Dsegments = ["GAAATAGT",               # D1
             "CCTTCCTAC",              # D2
             "ACTGGGGGATACG",          # D3
             "CCTTCCTACACTGGGGGATACG"  # D2-D3 merged
            ]
filepath = "./data/britain/vdj.10_TRD_full.txt"
df = pd.read_csv(filepath, sep = "\t")
df.head()

,#count,freq,cdr3nt,cdr3aa,v,d,j,VEnd,DStart,DEnd,JStart
0,123727,0.486201,TGTGCCTGCCGATTTGGGTACTGGGGGTCCACCGATAAACTCATCTTT,CACRFGYWGSTDKLIF,TRDV2,TRDD3,TRDJ1,7,19,26,29
1,45214,0.177674,TGTGCTCTTGGGGAACGTTCCCTTCCTGCCAAACGATATGGGGTGC...,CALGERSLPAKRYGVLGISLPYTDKLIF,TRDV1,TRDD3,TRDJ1,15,47,53,64
2,23672,0.093022,TGTGCCTCCCTAGGCCTTCCTACGGGGCTGGGGGAAAAACTCATCTTT,CASLGLPTGLGEKLIF,TRDV2,TRDD2,TRDJ1,6,14,22,33
3,19940,0.078357,TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAA...,CALGESPWRSNCVLKTTHLTAQLFF,TRDV1,.,TRDJ2,13,-1,-1,53
4,18225,0.071617,TGTGCTCTTGGGGAACGGAGAGGGTACTGGGGTCCCACCGATAAAC...,CALGERRGYWGPTDKLIF,TRDV1,TRDD3,TRDJ1,15,25,31,35


In [6]:
def cut_ends(a):
    res = ""
    for i in range(len(a[0])):
        if a[0][i] == a[1][i]:
            res += a[1][i]
        elif a[1][i] == "-":
            res += "-"
        else:
            res += "*"
    return (a[0], res, a[2], a[3])


def pretty(a):
    print(a[0])
    print(a[1])
    

def find_the_longest(seq, Dsegments, min_len):
    
    def get_start_end_len(a):
        start = -1
        end = -1
        
        max_start, max_end, max_len = -1, -1, -1
        for i in range(0, len(a[0])):
            if a[0][i] == "-" or a[1][i] == "-":
                if end - start + 1 > max_len:
                    max_start = start
                    max_end = end
                    max_len = end - start + 1
                start = -1
                end = -1
            else:
                if a[0][i] == a[1][i]:
                    if start == -1:
                        start = i
                    end = i
                else:
                    if end - start + 1 > max_len:
                        max_start = start
                        max_end = end
                        max_len = end - start + 1

                    start = -1
                    end = -1
        
        return max_start, max_end, max_len
    
    D_alignments = []
    for D in Dsegments:
        a = pairwise2.align.globalxs(seq, D, -3, -3, penalize_end_gaps = False)[0]
        cur_start, cur_end, cur_len = get_start_end_len(a)
        pretty(cut_ends(a))
        print(cur_start, cur_end, cur_len)
        D_alignments.append((cur_start, cur_end, cur_len, Dsegments.index(D)))
            
    D_alignments = sorted(D_alignments, reverse = True, key = lambda x: x[3])
    return list(filter(lambda x: x[2] >= min_len, sorted(D_alignments, reverse = True, key = lambda x: x[2])))


res = find_the_longest(df["cdr3nt"][3], Dsegments, 3)
print("Alignment:\t", res)
# if the best score is D2D3,
# check if D2D3 score better than D2 or D3
if res[0][3] == 3:
    if res[0][2] > res[1][2]:
        # if so then this is true D2D3 without N alignment
        pass
        # return smth
    else:
        # otherwise it's just one of the D2, D3 or D2-N-D3 alignment
        # so just pop it
        res.pop(0)

# if the best score isn't D2D3,
# then check for dual alignments


TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAACGACCCATTTGACAGCACAACTCTTCTTT
----------------------------*AAA**GT---------------------------------------
29 31 3
TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAACGACCCATTTGACAGCACAACTCTTCTTT-
-------------------------------------------------------------------*CTTC*T**
68 71 4
TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAACGACCCATTTGACAGCACAACTCTTCTTT
-----*CT*GGGGA*A**---------------------------------------------------------
9 13 5
----TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAACGACCCATTTGACAGCACAACTCTTCTTT
******T*C*CT*GGGGA*A**---------------------------------------------------------
13 17 5
Alignment:	 [(13, 17, 5, 3), (9, 13, 5, 2), (68, 71, 4, 1), (29, 31, 3, 0)]


In [54]:
# разбросанные по сиквенсу
seq = df["cdr3nt"][3]
print(df.head())
# seq = df["cdr3nt"][1]
# seq = df["cdr3nt"][0]
for D in Dsegments:
    a = pairwise2.align.globalms(seq, D, 2, -1, -20, -20, penalize_end_gaps = False)[0]
    pretty(cut_ends(a))
    print(a[2:])
    

   #count      freq                                             cdr3nt  \
0  123727  0.486201   TGTGCCTGCCGATTTGGGTACTGGGGGTCCACCGATAAACTCATCTTT   
1   45214  0.177674  TGTGCTCTTGGGGAACGTTCCCTTCCTGCCAAACGATATGGGGTGC...   
2   23672  0.093022   TGTGCCTCCCTAGGCCTTCCTACGGGGCTGGGGGAAAAACTCATCTTT   
3   19940  0.078357  TGTGCTCTTGGGGAGAGCCCCTGGAGGTCAAACTGTGTATTAAAAA...   
4   18225  0.071617  TGTGCTCTTGGGGAACGGAGAGGGTACTGGGGTCCCACCGATAAAC...   

                         cdr3aa      v      d      j  VEnd  DStart  DEnd  \
0              CACRFGYWGSTDKLIF  TRDV2  TRDD3  TRDJ1     7      19    26   
1  CALGERSLPAKRYGVLGISLPYTDKLIF  TRDV1  TRDD3  TRDJ1    15      47    53   
2              CASLGLPTGLGEKLIF  TRDV2  TRDD2  TRDJ1     6      14    22   
3     CALGESPWRSNCVLKTTHLTAQLFF  TRDV1      .  TRDJ2    13      -1    -1   
4            CALGERRGYWGPTDKLIF  TRDV1  TRDD3  TRDJ1    15      25    31   

   JStart  
0      29  
1      64  
2      33  
3      53  
4      35  
TGTGCTCTTGGGGAGAGCCCCTGGAG

In [26]:
seq1 = "TGTGCTTTT"
seq2 = "GTGTGGGGAGA"
a = pairwise2.align.localms(seq1, seq2, 2, -1, -3, -3, penalize_end_gaps = False)[0]
pretty(cut_ends(a))
print(a[2:])
    

-TGTGCTTTT-
*TGTG******
(8.0, 1, 5)
